<h1 align="center">การใช้งาน Spark ร่วมกับ Python</h1>

---

<img src="https://blog.datath.com/wp-content/uploads/2021/03/Pyspark.png" align="center" width="400">

<h3 align="right" style="color:blue;">Author: Pasit Y.</h3>

---

> ## Import Libraly ที่ต้องการใช้งาน (Spark, Pandas)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
from datetime import datetime
from urllib.request import urlopen

> ## ตั้งค่า Environment ที่จำเป็นลักษณะมีประมาณนี้โดยปกติจะใช้ได้เครื่อง

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'

conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '4g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     #('spark.sql.warehouse.dir", "/user/hive/warehouse'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("myApp") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

> ## อ่าน CSV Files

In [ ]:
myDF = spark.read.csv("file:///HDFS01/airflow/notebooks/Pasit/PySpark Tutorial/example.csv",
                      header=True, inferSchema=True)

> ## อ่าน Json From URL

In [ ]:
url = 'https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces'
jsonData = urlopen(url).read().decode('utf-8')
rdd = spark.sparkContext.parallelize([jsonData])
df = spark.read.json(rdd)
df2 = spark.read.json(rdd)

> ## สร้าง Cache View Table (Lazy) เพื่อมาทำการ Cleansing จะเร็วกว่า Inline 

In [ ]:
df = df.createOrReplaceTempView('covid_temp')
spark.sql('SELECT * FROM covid_temp').limit(10).toPandas()

> ## ตัวอย่างการ Filter แต่ละ Function

### แสดง Schema ของ Datasets หาก Spark ทำการ Create auto ให้แล้วไม่ตรงกับรูปแบบที่เราต้องการ มีความจำเป็นต้อง Manual Create ดังนี้

In [ ]:
#แสดง Schema ของ Datasets หาก Spark ทำการ Create auto ให้แล้วไม่ตรงกับรูปแบบที่เราต้องการ มีความจำเป็นต้อง Manual Create ดังนี้
#schema = StructType([
#    StructField("year", LongType(), True),
#    StructField("weeknum", LongType(), True),
#    StructField("province", StringType(), True),
#    StructField("new_case", LongType(), True),
#    StructField("new_case_excludeabroad", LongType(), True),
#    StructField("new_death", LongType(), True),
#    StructField("total_case", LongType(), True),
#    StructField("total_case_excludeabroad", LongType(), True),
#    StructField("total_death", LongType(), True),
#    StructField("update_date", StringType(), True)
#)]
myDF.printSchema()

### แสดงข้อมูลแบบเลือก Columns และ Limit

In [ ]:
df.select(['year','province']).limit(5).toPandas()

### ลบ Column ที่ไม่ได้ใช้งาน

In [ ]:
df = df.drop('total_death','weeknum')

### Filter เลือกข้อมูล (Where)

In [ ]:
df.filter(df.province == 'ระยอง').limit(10).toPandas()

### Group By same SQL

In [ ]:
df.groupby('province').count().limit(100).toPandas()

### Filter แบบหลายเงื่อนไข (&, |)

In [ ]:
df.select('province','update_date', 'new_case', 'total_case') \
    .filter( (df.province  == "ระยอง") | (df.province  == "ระนอง") ) \
    .limit(10) \
    .toPandas()

### To Date Function

In [ ]:
df = df.withColumn("update_date",to_date(col("update_date"))) \
  .show(truncate=False)

### รวม Column เป็น Column ใหม่ด้วย Sep (|,)

In [ ]:
df.withColumn("update_date",expr(" update_date ||'-'|| update_date")).toPandas()

### Case When Function

In [ ]:
df2 = df.withColumn("province", expr("""CASE
											WHEN province = 'ระยอง' 
                								THEN 'Pattani'
           									WHEN province = 'กรุงเทพมหานคร'
                                            	THEN 'Bangkok'
                                             ELSE 'unknown' END
           """))

### แทนที่ Special Character

In [ ]:
df = df.withColumn("details", regexp_replace("details", "\r\n", ""))

### สร้าง Columns ใหม่แบบมีเงื่อนไข

In [ ]:
df = df.withColumn('new_cols', lit(None).cast(StringType()))

### เปลี่ยนชื่อ Columns

In [ ]:
df = df.withColumnRenamed("new_case","newcase")

### ลบ Columns ที่ไม่ต้องการใช้ออก

In [ ]:
df = df.drop("province", "new_case", "update_date")

### ลบค่าที่ซ้ำกัน

In [ ]:
df = df.dropDuplicates()

### แทนที่ค่าว่างด้วย Assign Keyword

In [ ]:
#df.na.fill(value='ว่าง',subset=["new_cols"]).show()

#df.fillna(value=0)

df.na.fill('ไม่มีข้อมูล', 'new_cols')

### Join 2 Dataframe

In [ ]:
df3 = df.join(df2,
               df.province == df2.province,
               "inner").toPandas()

### Order By & Sort By
- OrderBy ASC .asc()
- OrderBy DESC .desc()

In [ ]:
df = df.sort(df.total_death.desc()).show(truncate=False)

### รวม Dataframe (Union)

In [ ]:
unionDF = df.union(df2)

---

<h1 align="center">HDFS Session</h1>

# เขียนข้อมูลลงไป HDFS
- <h3 style="color: red;"> 1. Basic Save ไม่ได้กำหนดค่าอะไรค่าเริ่มต้นจะเป็น PARQUET Type</h3>
- ### <h3 style="color: blue;"> 2. Format Static กำหนดค่ารูปแบบที่จะไปจัดเก็บใน HDFS มี 2 รูปแบบคือ PARQUET, ORC </h3>
- ###  <h3 style="color: green;">3. กำหนดรูปแบบการเก็บแบบ Partition รูปแบบการเก็บแบบแยก schema/tablename/partiton ข้อดีคือจะมีความรวดเร็วในการ Insert เนื่องจากรูปแบบการเก็บชัดเจนหาก Data มาในชนิดเดียวกัน แต่หากมีการเก็บข้อมูลที่ไม่เหมือนกันหรือต่าง Data Source กัน อาจจะต้องการ Repartition ก่อนการ Save เพื่อเพิ่มหรือลดพาร์ติชัน RDD/DataFrame แต่ถ้าต้องการลดเท่านั้นให้ใช้ coalesce() เพื่อลดจำนวนพาร์ติชันอย่างมีประสิทธิภาพ </h3>
- ### <h3 style="color: #D35400;">4. กำหนดรูปแบบการเก็บลักษณะ Partition -> จำนวนไฟล์ที่ Split ออกมาในลักษณะ schema/tablename/partiton/bucket (ตามจำนวน bucket ที่ระบุไว้) เพื่อให้มีขนาดไฟล์ที่เล็กเพื่อความเร็วในการ Query </h3>
---
# ข้อดีและข้อเสีย
- ### PartitonBy 
  * ### ข้อดีคือ มีความเร็วในการจัดเก็บข้อมูลสูงทั้ง Overwrite / Append เน้นเขียนซ้ำเพิ่มลดจำนวนบ่อยครั้ง (ETL High Performance)
  * ### ข้อเสียคือ ความเร็วในการ Query ยังสู้ BucketBy ไม่ได้ เนื่องจาก Partiton มันคือการ Group เพื่อให้มีการลดความยุ่งยากค้นหาของ HDFS (MapReduce)
- ### BucketBy
  * ### ข้อดีคือ มีการ Query ที่รวดเร็วกว่า PartitionBy เนื่องจาก Bucket จะทำการ Split Large File ให้เป็นไฟล์เล็กๆหลายๆไฟล์ ทำให้การ Reduce ข้อมูลได้มีความรวดเร็วมากกว่า
  * ### ข้อเสียคือ ไม่เหมาะกับข้อมูลที่เป็นรูปแบบ Incremental หรือมีข้อมูลเข้ามาตลอดเวลาและ Overwrite / Append ช้ากว่า PartitionBy
---
# MapReduce คืออะไร

> ## แยกเป็นรูปแบบการทำงานดังนี้
* ### Map คือ การเอาข้อมูลขนาดใหญ่มาจัดให้อยู่ในรูปแบบของ Key => Value หลังจากนั้นมาทำการ Shuffle ให้กระจายออกมาเป็นหลายๆไฟล์ และการะจายการจัดเก็บไปยัง Worker ต่างๆใน Cluster
* ### Reduce ตือการนำข้อมูลที่ถูก Map Key => Value และ Shuffle จากหลายๆเครื่องมารวมกันเพื่อให้ได้มาเป็น Object 1 ก้อน เมื่อเวลาเรา Query ข้อมูล ดังภาพ

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*WlJFdJP108MTvR8j-S0UGA.png">

### อ่านด้วย read.table ให้ระบุเป็น schema.tablename

In [ ]:
myDF = spark.read.table("pyspark.j64_ew_status_ability")

### อ่านข้อมูลโดยใช้ SQL Statement

In [ ]:
myDF2 = spark.sql("SELECT * FROM pyspark.j64_ew_status_ability LIMIT 10")

### อ่านข้อมูลแบบใช้ Filter

In [ ]:
myDF3 = spark.read.table("pyspark.j64_ew_status_ability") \
       .filter( (myDF.Group_ew  == "Mobile") | (myDF.Status  == "100") ) \
       .limit(10) \
       .toPandas()

<h3 style="color: red;"> เขียนข้อมูลไปยัง HDFS (ข้อที่ 1)</h3>

In [ ]:
#1
df.write \
    .mode("overwrite") \
    .saveAsTable("pyspark.covid_temp")

<h3 style="color: blue;"> เขียนข้อมูลไปยัง HDFS (ข้อที่ 2)</h3>

In [ ]:
#2
df.write \
    .format("parquet") \
    .mode("overwrite") \
    .saveAsTable("pyspark.covid_temp")

<h3 style="color: green;"> เขียนข้อมูลไปยัง HDFS (ข้อที่ 3)</h3>

In [ ]:
#3
df = df.repartition(6) #ตัวเลขที่ต้องการเพิ่ม/ลด จำนวน Partition
df = df.coalesce(4) ##ตัวเลขที่ต้องการลด จำนวน Partition
df.write \
    .mode("overwrite") \
    .partitionBy("province") \
    .saveAsTable("pyspark.covid_temp")

<h3 style="color: #D35400;"> เขียนข้อมูลไปยัง HDFS (ข้อที่ 4)</h3>

In [ ]:
#4
df = df.repartition(6) #ตัวเลขที่ต้องการเพิ่ม/ลด จำนวน Partition
df = df.coalesce(4) ##ตัวเลขที่ต้องการลด จำนวน Partition
df.write \
    .mode("overwrite") \
    .partitionBy("province") \
    .bucketBy(10, "year")
    .saveAsTable("pyspark.covid_temp")

In [ ]:
#2
myDF.write \
    .format("parquet") \
    .mode("overwrite") \
    .saveAsTable("pyspark.covid_temp3")

In [ ]:
os.system('sudo -u hdfs hdfs dfs -ls /user/hive/warehouse/iceberg_test/')